## Problem Statement
In this project, we have to predict the number of positive and negative reviews based on sentiments by using different classification models.

### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer















KeyboardInterrupt: 